In [2]:

from utils_aa import * 

study_real = optuna.create_study(
    study_name='study_real',
    storage="sqlite:///alejandro.db",
    directions=["maximize", "maximize"],
    # sampler=optuna.samplers.TPESampler(),  # NSGAIISampler(),
    sampler=optuna.samplers.NSGAIISampler(), # NSGAIISampler(),
    load_if_exists=True, 
)


study_real_sampling = optuna.create_study(
    study_name='study_real_sampling',
    storage="sqlite:///alejandro.db",
    directions=["maximize", "maximize"],
    # sampler=optuna.samplers.TPESampler(),  # NSGAIISampler(),
    sampler=optuna.samplers.NSGAIISampler(), # NSGAIISampler(),
    load_if_exists=True,
)



make_pareto_plot(study_real_sampling)


[I 2023-05-05 10:31:26,026] Using an existing study with name 'study_real' instead of creating a new one.
[I 2023-05-05 10:31:26,043] Using an existing study with name 'study_real_sampling' instead of creating a new one.


In [3]:

make_ranking_plots(study_real_sampling, get_median_dict(study_real), data ="_best_artifact/optuna/data.csv", CUTOFF_AUC =.9, CUTOFF_PHE=.0)

{'best_iteration': '39', 'best_ntree_limit': '40', 'best_score': '0.08114367102583249'}
Phe_value :  0.15181865 	 Phe_ranking :  15.0


{'best_iteration': '2', 'best_ntree_limit': '3', 'best_score': '0.42957672476768494'}
Phe_value :  0.21201801 	 Phe_ranking :  8.0


{'best_iteration': '20', 'best_ntree_limit': '21', 'best_score': '0.21961123868823051'}
Phe_value :  0.25939038 	 Phe_ranking :  7.0


{'best_iteration': '22', 'best_ntree_limit': '23', 'best_score': '0.3154486215983828'}
Phe_value :  0.30960056 	 Phe_ranking :  5.0


{'best_iteration': '14', 'best_ntree_limit': '15', 'best_score': '0.35881624184548855'}
Phe_value :  0.22882034 	 Phe_ranking :  9.0


{'best_iteration': '11', 'best_ntree_limit': '12', 'best_score': '0.4599017488459746'}
Phe_value :  0.36585155 	 Phe_ranking :  6.0


{'best_iteration': '25', 'best_ntree_limit': '26', 'best_score': '0.1610239134170115'}
Phe_value :  0.25733283 	 Phe_ranking :  10.0


{'best_iteration': '4', 'best_ntree_limit': '5', 'best_sco

# ----------------------------------------------------

In [4]:

#study_name: str = "alejandro_resampled_data_BorderlineSMOTE"

study_BorderlineSMOTE = optuna.create_study(
    study_name='study_BorderlineSMOTE',
    storage="sqlite:///alejandro.db",
    directions=["maximize", "maximize"],
    # sampler=optuna.samplers.TPESampler(),  # NSGAIISampler(),
    sampler=optuna.samplers.NSGAIISampler(), # NSGAIISampler(),
    load_if_exists=True,
)

study_SVMSMOTE = optuna.create_study(
    study_name='study_SVMSMOTE',
    storage="sqlite:///alejandro.db",
    directions=["maximize", "maximize"],
    # sampler=optuna.samplers.TPESampler(),  # NSGAIISampler(),
    sampler=optuna.samplers.NSGAIISampler(), # NSGAIISampler(),
    load_if_exists=True,
)


study_ADASYN = optuna.create_study(
    study_name='study_ADASYN',
    storage="sqlite:///alejandro.db",
    directions=["maximize", "maximize"],
    # sampler=optuna.samplers.TPESampler(),  # NSGAIISampler(),
    sampler=optuna.samplers.NSGAIISampler(), # NSGAIISampler(),
    load_if_exists=True,
)

study_real = optuna.create_study(
    study_name='study_real',
    storage="sqlite:///alejandro.db",
    directions=["maximize", "maximize"],
    # sampler=optuna.samplers.TPESampler(),  # NSGAIISampler(),
    sampler=optuna.samplers.NSGAIISampler(), # NSGAIISampler(),
    load_if_exists=True, 
)

# ########################
# study_real.optimize(lambda trial: objective(trial, "_best_artifact/optuna/data.csv",                                                        finetunning = True), n_trials=5000, n_jobs=-1)
# study_BorderlineSMOTE.optimize(lambda trial: objective(trial, "_best_artifact/optuna/resampled_data_BorderlineSMOTE.csv",                   finetunning = True), n_trials=5000, n_jobs=-1)
# study_SVMSMOTE.optimize(lambda trial: objective(trial, "_best_artifact/optuna/resampled_data_SVMSMOTE.csv",                                 finetunning = True), n_trials=5000, n_jobs=-1)
# study_ADASYN.optimize(lambda trial: objective(trial, "_best_artifact/optuna/resampled_data_ADASYN.csv",                                     finetunning = True), n_trials=5000, n_jobs=-1)
# ###################
#make_pareto_plot(study_real)


[I 2023-05-05 10:31:34,230] Using an existing study with name 'study_BorderlineSMOTE' instead of creating a new one.
[I 2023-05-05 10:31:34,247] Using an existing study with name 'study_SVMSMOTE' instead of creating a new one.
[I 2023-05-05 10:31:34,264] Using an existing study with name 'study_ADASYN' instead of creating a new one.
[I 2023-05-05 10:31:34,281] Using an existing study with name 'study_real' instead of creating a new one.


In [5]:

######## SAMPLING ########

study_BorderlineSMOTE_sampling = optuna.create_study(
    study_name='study_BorderlineSMOTE_sampling',
    storage="sqlite:///alejandro.db",
    directions=["maximize", "maximize"],
    # sampler=optuna.samplers.TPESampler(),  # NSGAIISampler(),
    sampler=optuna.samplers.NSGAIISampler(), # NSGAIISampler(),
    load_if_exists=True,
)

study_SVMSMOTE_sampling = optuna.create_study(
    study_name='study_SVMSMOTE_sampling',
    storage="sqlite:///alejandro.db",
    directions=["maximize", "maximize"],
    # sampler=optuna.samplers.TPESampler(),  # NSGAIISampler(),
    sampler=optuna.samplers.NSGAIISampler(), # NSGAIISampler(),
    load_if_exists=True,
)



study_ADASYN_sampling = optuna.create_study(
    study_name='study_ADASYN_sampling',
    storage="sqlite:///alejandro.db",
    directions=["maximize", "maximize"],
    # sampler=optuna.samplers.TPESampler(),  # NSGAIISampler(),
    sampler=optuna.samplers.NSGAIISampler(), # NSGAIISampler(),
    load_if_exists=True,
)

study_real_sampling = optuna.create_study(
    study_name='study_real_sampling',
    storage="sqlite:///alejandro.db",
    directions=["maximize", "maximize"],
    # sampler=optuna.samplers.TPESampler(),  # NSGAIISampler(),
    sampler=optuna.samplers.NSGAIISampler(), # NSGAIISampler(),
    load_if_exists=True,
)


[I 2023-05-05 10:31:34,370] Using an existing study with name 'study_BorderlineSMOTE_sampling' instead of creating a new one.
[I 2023-05-05 10:31:34,390] Using an existing study with name 'study_SVMSMOTE_sampling' instead of creating a new one.
[I 2023-05-05 10:31:34,407] Using an existing study with name 'study_ADASYN_sampling' instead of creating a new one.
[I 2023-05-05 10:31:34,426] Using an existing study with name 'study_real_sampling' instead of creating a new one.


In [22]:



# study_real_sampling.optimize(lambda trial: objective(trial, "_best_artifact/optuna/data.csv",  tuned_params = get_median_dict(study_real),                    finetunning = False), n_trials=1000, n_jobs=-1)
# study_ADASYN_sampling.optimize(lambda trial: objective(trial, "_best_artifact/optuna/resampled_data_ADASYN.csv",  tuned_params = get_median_dict(study_ADASYN),                    finetunning = False), n_trials=1000, n_jobs=-1)
# study_BorderlineSMOTE_sampling.optimize(lambda trial: objective(trial, "_best_artifact/optuna/resampled_data_BorderlineSMOTE.csv",  tuned_params = get_median_dict(study_BorderlineSMOTE),  finetunning = False), n_trials=1000, n_jobs=-1)
# study_SVMSMOTE_sampling.optimize(lambda trial: objective(trial, "_best_artifact/optuna/resampled_data_SVMSMOTE.csv",  tuned_params = get_median_dict(study_SVMSMOTE),                finetunning = False), n_trials=1000, n_jobs=-1)
import optuna
from optuna.trial import TrialState

def get_filtered_study(original_study,objective_index, lower_bound):
    
    #original_study = study_real_sampling

    # Function to filter trials based on the value of one objective
    def filter_trials_by_objective(study, objective_index, value_range):
        filtered_trials = []
        for trial in study.get_trials():
            if trial.state == optuna.trial.TrialState.COMPLETE:
                value = trial.values[objective_index]
                if value_range[0] <= value <= value_range[1]:
                    filtered_trials.append(trial)
        return filtered_trials

    # Filter trials with the first objective value in the range [lower_bound, upper_bound]
    #objective_index = 1
   #lower_bound = 0.2
    upper_bound = 1
    filtered_trials = filter_trials_by_objective(original_study, objective_index, (lower_bound, upper_bound))

    # Create a new study with the filtered trials
    filtered_study = optuna.create_study(directions=["minimize", "minimize"])
    for trial in filtered_trials:
        filtered_study.add_trial(trial)


    return filtered_study



In [30]:
filtered_study_sampling = get_filtered_study(study_real_sampling, 1, 0.2)

make_pareto_plot(filtered_study_sampling)
make_ranking_plots(filtered_study_sampling, get_median_dict(study_real), data ="_best_artifact/optuna/data.csv", CUTOFF_AUC =.9, CUTOFF_PHE=.0)

[I 2023-05-05 12:38:46,048] A new study created in memory with name: no-name-448c51e5-ac27-4269-81d1-fb7affc0f2a8


{'best_iteration': '2', 'best_ntree_limit': '3', 'best_score': '0.42957672476768494'}
Phe_value :  0.21201801 	 Phe_ranking :  8.0


{'best_iteration': '20', 'best_ntree_limit': '21', 'best_score': '0.21961123868823051'}
Phe_value :  0.25939038 	 Phe_ranking :  7.0


{'best_iteration': '22', 'best_ntree_limit': '23', 'best_score': '0.3154486215983828'}
Phe_value :  0.30960056 	 Phe_ranking :  5.0


{'best_iteration': '14', 'best_ntree_limit': '15', 'best_score': '0.35881624184548855'}
Phe_value :  0.22882034 	 Phe_ranking :  9.0


{'best_iteration': '11', 'best_ntree_limit': '12', 'best_score': '0.4599017488459746'}
Phe_value :  0.36585155 	 Phe_ranking :  6.0


{'best_iteration': '25', 'best_ntree_limit': '26', 'best_score': '0.1610239134170115'}
Phe_value :  0.25733283 	 Phe_ranking :  10.0


{'best_iteration': '4', 'best_ntree_limit': '5', 'best_score': '0.42750075459480286'}
Phe_value :  0.29135025 	 Phe_ranking :  6.0


{'best_iteration': '16', 'best_ntree_limit': '17', 'best_scor

In [33]:
#make_pareto_plot(study_ADASYN_sampling)
#make_ranking_plots(study_ADASYN_sampling, get_median_dict(study_ADASYN), "_best_artifact/optuna/resampled_data_ADASYN.csv", CUTOFF_AUC =.9, CUTOFF_PHE=.3)



filtered_study_sampling = get_filtered_study(study_ADASYN_sampling, 0, 0.5)

make_pareto_plot(filtered_study_sampling)
make_ranking_plots(filtered_study_sampling, get_median_dict(study_ADASYN), data ="_best_artifact/optuna/data.csv", CUTOFF_AUC =.9, CUTOFF_PHE=.0)

[I 2023-05-05 12:39:58,908] A new study created in memory with name: no-name-997cc996-5ab2-41e4-a010-40386ff338e6


/DeepenData/Repos/Phenylketonuria_IR_and_Machine_Learning/utils_aa.py:339: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/DeepenData/Repos/Phenylketonuria_IR_and_Machine_Learning/utils_aa.py:339: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/DeepenData/Repos/Phenylketonuria_IR_and_Machine_Learning/utils_aa.py:339: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.

{'best_iteration': '20', 'best_ntree_limit': '21', 'best_score': '0.2872977244357268'}
Phe_value :  0.0 	 Phe_ranking :  30.0


{'best_iteration': '12', 'best_ntree_limit': '13', 'best_score': '0.2391181712349256'}
Phe_value :  0.0 	 Phe_ranking :  27.0


{'best_iteration': '25', 'best_ntree_limit': '26', 'best_score': '0.28660963475704193'}
Phe_value :  0.0 	 Phe_ranking :  28.5


{'best_iteration': '11', 'best_ntree_limit': '12', 'best_score': '0.42539931647479534'}
Phe_value :  0.20659575 	 Phe_ranking :  8.0


{'best_iteration': '13', 'best_ntree_limit': '14', 'best_score': '0.39492135184506577'}
Phe_value :  0.0 	 Phe_ranking :  27.0


{'best_iteration': '17', 'best_ntree_limit': '18', 'best_score': '0.382334624727567'}
Phe_value :  0.0 	 Phe_ranking :  29.0


{'best_iteration': '12', 'best_ntree_limit': '13', 'best_score': '0.364143871391813'}
Phe_value :  0.17588574 	 Phe_ranking :  8.0


{'best_iteration': '6', 'best_ntree_limit': '7', 'best_score': '0.3217802668611209'}
Phe_va

/DeepenData/Repos/Phenylketonuria_IR_and_Machine_Learning/utils_aa.py:371: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/DeepenData/Repos/Phenylketonuria_IR_and_Machine_Learning/utils_aa.py:371: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



{'best_iteration': '4', 'best_ntree_limit': '5', 'best_score': '0.39030235012372333'}
Phe_value :  0.0 	 Phe_ranking :  26.5


{'best_iteration': '6', 'best_ntree_limit': '7', 'best_score': '0.3597908864418666'}
Phe_value :  0.0 	 Phe_ranking :  28.5




...

/DeepenData/Repos/Phenylketonuria_IR_and_Machine_Learning/utils_aa.py:371: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/DeepenData/Repos/Phenylketonuria_IR_and_Machine_Learning/utils_aa.py:371: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



{'best_iteration': '5', 'best_ntree_limit': '6', 'best_score': '0.4798685734470685'}
Phe_value :  0.0 	 Phe_ranking :  25.5


{'best_iteration': '36', 'best_ntree_limit': '37', 'best_score': '0.23970229675372443'}
Phe_value :  0.22384767 	 Phe_ranking :  7.0




/DeepenData/Repos/Phenylketonuria_IR_and_Machine_Learning/utils_aa.py:371: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/DeepenData/Repos/Phenylketonuria_IR_and_Machine_Learning/utils_aa.py:371: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



{'best_iteration': '4', 'best_ntree_limit': '5', 'best_score': '0.5328324486811956'}
Phe_value :  0.0 	 Phe_ranking :  26.0




/DeepenData/Repos/Phenylketonuria_IR_and_Machine_Learning/utils_aa.py:371: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/DeepenData/.miniconda/envs/playground/lib/python3.10/site-packages/plotly/express/_core.py:1223: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/DeepenData/.miniconda/envs/playground/lib/python3.10/site-packages/plotly/express/_core.py:1223: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all co

In [ ]:
make_pareto_plot(study_BorderlineSMOTE_sampling)
make_ranking_plots(study_BorderlineSMOTE_sampling, get_median_dict(study_BorderlineSMOTE),data="_best_artifact/optuna/resampled_data_BorderlineSMOTE.csv" ,CUTOFF_AUC =.9, CUTOFF_PHE=.3)



In [ ]:

make_pareto_plot(study_SVMSMOTE_sampling)
make_ranking_plots(study_SVMSMOTE_sampling, get_median_dict(study_SVMSMOTE), data="_best_artifact/optuna/resampled_data_SVMSMOTE.csv",CUTOFF_AUC =.9, CUTOFF_PHE=.1)